In [1]:
import shapely
from shapely import Point, LineString, Polygon, MultiPolygon
import os
import geopandas as gpd
import pandas as pd
import math

In [31]:
# calculates great circle (length) distance with a forward point
def coords_length(phi1,phi2,lam1,lam2):
    phi1 = math.radians(phi1) 
    phi2 = math.radians(phi2)
    lam1 = math.radians(lam1)
    lam2 = math.radians(lam2)
    dl = abs(lam1-lam2) 
    # distance calculation
    d = math.acos(min(math.sin(phi1) * math.sin(phi2) + math.cos(phi1) * math.cos(phi2) * math.cos(dl), 1.0))
    spheric_dist = d*6378.137*1000
    # spheric_dist in meters
    return spheric_dist

# calculates angle formed with a forward point, current and backward point
def calculate_angle(x1, x2, x3, y1, y2, y3):
    # Calculate the vectors between the points
    v1 = (x1 - x2, y1 - y2)
    v2 = (x3 - x2, y3 - y2)
    # Calculate the dot product and the magnitudes of the vectors
    dot_product = v1[0] * v2[0] + v1[1] * v2[1]
    magnitude_v1 = math.sqrt(v1[0] ** 2 + v1[1] ** 2)
    magnitude_v2 = math.sqrt(v2[0] ** 2 + v2[1] ** 2)
    # Check if the dot product and magnitudes are within valid ranges
    if magnitude_v1 * magnitude_v2 == 0:
        return float("NaN")
    # Calculate the cosine of the angle using the dot product and magnitudes
    cosine_angle = dot_product / (magnitude_v1 * magnitude_v2)
    # Ensure the value is within the valid range for acos function
    cosine_angle = max(min(cosine_angle, 1.0), -1.0)
    # Calculate the angle in radians
    angle_rad = math.acos(cosine_angle)
    # Convert the angle from radians to degrees
    angle_deg = math.degrees(angle_rad)
    return angle_deg

def calculate_offset(x1,x2,x3,y1,y2,y3):
    # Calculate the slope of the line
    if (x3-x1) != 0:
        m = (y3-y1) / (x3-x1)
    else:
        return float("NaN")
    # Calculate the y-intercept of the line
    c = y1 - (m * x1)
    # Calculate the perpendicular distance from the point (x2, y2) to the line
    ppdistance = abs((m * x2 - y2) + c) / math.sqrt(m ** 2 + 1)
    return (ppdistance*6378.137*1000)

def calculate_triangluar_area(x1,x2,x3,y1,y2,y3):
    # Calculate the area of the triangle using the Shoelace formula
    area = 0.5 * abs((x1 * (y2 - y3)) + (x2 * (y3 - y1)) + (x3 * (y1 - y2)))
    return (area*6378.137*1000)

# calculates angle formed with a forward point
def calculate_arc_angle(x1,x2,y1,y2):
    import math
    x1 = math.radians(x1) 
    x2 = math.radians(x2)
    y1 = math.radians(y1)
    y2 = math.radians(y2)
    # angle calculation
    angle = math.atan2(y2-y1, x2-x1)*180/math.pi
    # angle in degree radians
    return angle

In [32]:
for file in os.listdir("Vertices_Labels"):
    if file.endswith("csv") == True:
        file_name = file.split("_")
        if os.path.exists(f"Vertices_Labels/{file_name[0]}.parq") != True:
            state_df = pd.read_csv(os.path.join("Vertices_Labels",file))
            state_df = state_df.drop(columns = ["Unnamed: 0"])

            # Create an empty column with name "len_forward"
            state_df["len_forward"] = 0.0
            for i in range(len(state_df)-1):
                # assign data to arguments
                phi1 = state_df.loc[i, "Longitude"] 
                phi2 = state_df.loc[i+1, "Longitude"] 
                lam1 = state_df.loc[i, "Latitude"]
                lam2 = state_df.loc[i+1, "Latitude"]
                # calculate length and append to a list
                state_df.loc[i,"len_forward"] = coords_length(phi1,phi2,lam1,lam2)

            # Assign NaN to the last row since there is no next row
            state_df.loc[len(state_df)-1, "len_forward"] = float("NaN")

            # Create an empty column with name "len_backward"
            state_df["len_backward"] = 0.0
            for i in range(len(state_df)-1, 0, -1):
                # assign data to arguments
                phi1 = state_df.loc[i, "Longitude"] 
                phi2 = state_df.loc[i-1, "Longitude"] 
                lam1 = state_df.loc[i, "Latitude"]
                lam2 = state_df.loc[i-1, "Latitude"]
                # calculate length and append to a list
                state_df.loc[i, "len_backward"] = coords_length(phi1,phi2,lam1,lam2)

            # Assign NaN to the last row since there is no next row
            state_df.loc[0,"len_backward"] = float("NaN")

            # Create an empty column "angle"
            state_df["angle"] = 0.0
            # Iterate over the rows of the DataFrame
            for i in range(1, len(state_df) - 1):
                # Get the coordinates of the three points
                x1 = state_df.loc[i-1, "Longitude"]
                y1 = state_df.loc[i-1, "Latitude"]
                x2 = state_df.loc[i, "Longitude"]
                y2 = state_df.loc[i, "Latitude"]
                x3 = state_df.loc[i+1, "Longitude"]
                y3 = state_df.loc[i+1, "Latitude"]

                # Assign the calculated angle to the 'angle' column of the current row
                state_df.loc[i,"angle"] = calculate_angle(x1,x2,x3,y1,y2,y3)

            # Create an empty column "offset"
            state_df["offset"] = 0.0
            # Iterate over the rows of the DataFrame
            for i in range(1, len(state_df) - 1):
                # Get the coordinates of the three points
                x1 = state_df.loc[i-1, "Longitude"]
                y1 = state_df.loc[i-1, "Latitude"]
                x2 = state_df.loc[i, "Longitude"]
                y2 = state_df.loc[i, "Latitude"]
                x3 = state_df.loc[i+1, "Longitude"]
                y3 = state_df.loc[i+1, "Latitude"]

                # Assign the calculated angle to the 'angle' column of the current row
                state_df.loc[i,"offset"] = calculate_offset(x1,x2,x3,y1,y2,y3)

            # Create an empty column "area"
            state_df["area"] = 0.0
            # Iterate over the rows of the DataFrame
            for i in range(1, len(state_df) - 1):
                # Get the coordinates of the three points
                x1 = state_df.loc[i-1, "Longitude"]
                y1 = state_df.loc[i-1, "Latitude"]
                x2 = state_df.loc[i, "Longitude"]
                y2 = state_df.loc[i, "Latitude"]
                x3 = state_df.loc[i+1, "Longitude"]
                y3 = state_df.loc[i+1, "Latitude"]

                # Assign the calculated angle to the 'angle' column of the current row
                state_df.loc[i,"area"] = calculate_triangluar_area(x1,x2,x3,y1,y2,y3)

            # Create an empty column with name "arc"
            state_df["arc"] = 0.0
            for i in range(len(state_df)-1):
                # assign data to arguments
                x1 = state_df.loc[i, "Longitude"] 
                x2 = state_df.loc[i+1, "Longitude"] 
                y1 = state_df.loc[i, "Latitude"]
                y2 = state_df.loc[i+1, "Latitude"]
                state_df.loc[i,"arc"] = calculate_arc_angle(x1,x2,y1,y2)

            # Create an empty column with name "ratio1"
            state_df["ratio1"] = 0.0

            for i in range(len(state_df)):
                if state_df.loc[i, "arc"] != 0:
                    state_df.loc[i, "ratio1"] =  (state_df.loc[i,"len_forward"] + state_df.loc[i,"len_backward"])/state_df.loc[i, "arc"]
                else:
                    state_df.loc[i, "ratio1"] = 0.0

            # Create an empty column with name "ratio1"
            state_df["ratio2"] = 0.0
            for i in range(len(state_df)):
                if state_df.loc[i, "arc"] != 0:
                    state_df.loc[i, "ratio2"] = state_df.loc[i,"offset"]/state_df.loc[i,"arc"]
                else:
                    state_df.loc[i, "ratio2"] = 0.0

            state_df["len_backward"].fillna(state_df["len_backward"].median(), inplace=True)
            state_df["area"].fillna(state_df["area"].median(), inplace=True)
            state_df["ratio1"].fillna(state_df["ratio1"].median(), inplace=True)

            state_df.to_parquet(f"Vertices_Labels/{file_name[0]}.parq")
            print(f"Vertices_Labels/{file_name[0]}.parq")
        else:
            print(f"Vertices_Labels/{file_name[0]}.parq already exists")

Vertices_Labels/California.parq already exists
Vertices_Labels/Florida.parq
Vertices_Labels/Idaho.parq already exists
Vertices_Labels/Louisiana.parq
Vertices_Labels/Maine.parq
Vertices_Labels/NorthCarolina.parq
Vertices_Labels/Texas.parq


In [ ]:
 for file in os.listdir("Vertices_Labels"):
    if file.endswith("parq") == True:
        state_df = pd.read_parquet(os.path.join("Vertices_Labels",file))
        points_list = [(row.Longitude, row.Latitude) for row in state_df.itertuples()] 
        points_gdf = gpd.GeoDataFrame(data = state_df, geometry = [Point(p) for p in points_list])
        points_gdf["buffer"] = points_gdf.geometry.buffer(0.4)
        
        sindex = points_gdf.sindex

        # Iterate over the buffers and find all points that fall within each buffer
        within_lists = []
        for buffer in points_gdf["buffer"]:
            # Find the indices of all points that intersect the buffer
            possible_matches_index = list(sindex.intersection(buffer.bounds))
            # Filter the candidate points by testing if they're actually within the buffer
            possible_matches = points_gdf.iloc[possible_matches_index]
            matches = possible_matches[possible_matches.intersects(buffer)]
            # Append the matches to the output list
            within_lists.append(list(matches.geometry))

        # Create a new GeoDataFrame with the results
        result_gdf = gpd.GeoDataFrame(geometry=[buffer for buffer in points_gdf["buffer"]])
        result_gdf["within"] = within_lists
        
        result_gdf["point_counts"] = result_gdf["within"].apply(lambda x: len(x))
        result_gdf["buffer"] = result_gdf["geometry"]
        
        state_final_gdf = points_gdf.merge(result_gdf, on="buffer")
        
        out_df = pd.DataFrame(state_final_gdf[["Longitude", "Latitude", "len_forward",
                                                   "len_backward","angle", "arc",
                                                   "area","point_counts", "offset",
                                                   "ratio1", "ratio2", "case"]])
        file_name = file.split("_")
        out_df.to_parquet(f"Vertices_Labels/{file_name[0]}.parq")
        print(f"Vertices_Labels/{file_name[0]}.parq")